In [ ]:
!bash drive/My\ Drive/datawithcred.sh

In [ ]:
from pathlib import Path
import tensorflow as tf
from src.data.make_dataset import GetDataset
from src.core.cloud import CloudTrainer

In [ ]:
tf.random.set_seed(123)

In [ ]:
dt = GetDataset(image_dims=(224, 224))

In [ ]:
train = dt.make_train_data(Path("chest_xray/train"))

In [ ]:
val = dt.make_train_data(Path("chest_xray/test"))

In [ ]:
trainer = CloudTrainer(experiment_name="VGGNet16-transfer", save_freq=30)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=./drive/My\ Drive/models --port=6006

In [ ]:
def get_model():
    vgg = tf.keras.applications.VGG16(include_top=False, 
                                        weights="imagenet", 
                                        input_shape=(dt.width, dt.height, 3))
    model = tf.keras.models.Sequential()
    for layer in vgg.layers:
        layer.trainable = False
        model.add(layer)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(4096, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(4096, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    return model

In [ ]:
def get_model():
    vgg = tf.keras.applications.VGG16(include_top=False, 
                                    weights="imagenet", 
                                    input_shape=(dt.width, dt.height, 3))
    model = tf.keras.models.Sequential()
    found_conv_layer = False
    for layer in vgg.layers:
        if not found_conv_layer:
            layer.trainable = False
        else:
            layer.trainable = True
        model.add(layer)
        if layer.name == "block4_conv3":
            found_conv_layer = True
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(4096, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(4096, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    return model

In [ ]:
trainer.add_keras_seq_model(get_model)

In [ ]:
trainer.add_callbacks([
        tf.keras.callbacks.ReduceLROnPlateau(verbose=1)
])

In [ ]:
trainer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="binary_crossentropy",
    metrics=["accuracy",
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

In [ ]:
trainer.fit(
    train,
    validation_data=val,
    epochs=30
)

In [ ]:
trainer.model.summary()

In [ ]:
trainer.model.save("drive/My Drive/models/VGGNet16-transfer/final_model.h5")

In [ ]:
train = dt.make_balaced_dataset(majority_class="chest_xray/train/PNEUMONIA",
                                minority_class="chest_xray/train/NORMAL",
                                transformations=[
                                    dt.image_flip_left_right,
                                    dt.image_central_crop,
                                    dt.image_adjust_saturation
                                ])

In [ ]:
aug_trainer = CloudTrainer(experiment_name="VGGNet16-balanced-tl2ft4", save_freq=100)

In [ ]:
aug_trainer.add_keras_seq_model(get_model)

In [ ]:
aug_trainer.add_callbacks([
        tf.keras.callbacks.ReduceLROnPlateau(verbose=1)
])

In [ ]:
aug_trainer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy",
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

In [ ]:
aug_trainer.fit(
    train,
    validation_data=val,
    epochs=50
)

In [ ]:
aug_trainer.model.save("drive/My Drive/models/VGGNet16-balanced-tl2/final_model.h5")